In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

In [3]:

# ----------------- Load Data -----------------
df = pd.read_excel("Solardata_final_1.xlsx")
df["Date and Time"] = pd.to_datetime(df["Date and Time"])
df["hour"] = df["Date and Time"].dt.hour
df["dayofweek"] = df["Date and Time"].dt.dayofweek
df["month"] = df["Date and Time"].dt.month

In [4]:
target = "Output Power (kW)"
X = df.drop(columns=["SL No.", "Date and Time", target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# ----------------- Preprocessing -----------------
num_cols = X.select_dtypes(include=["number"]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

In [7]:
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("power", PowerTransformer()),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, num_cols),
    ("cat", categorical_transformer, cat_cols)
])

In [8]:
# ----------------- Model -----------------
xgb_pipeline = Pipeline([
    ("preproc", preprocessor),
    ("xgb", xgb.XGBRegressor(random_state=42, n_jobs=-1, verbosity=0))
])

xgb_pipeline.fit(X_train, y_train)
preds = xgb_pipeline.predict(X_test)

In [10]:
print("MAE:", mean_absolute_error(y_test, preds))
print("RMSE:", mean_squared_error(y_test, preds))
print("R²:", r2_score(y_test, preds))

MAE: 11.280904998513526
RMSE: 406.13922358351954
R²: 0.9999896803269812
